In [1]:
!python -V


Python 3.6.8


In [2]:
import spacy
from bs4 import BeautifulSoup
import json

In [3]:
with open('datasets/foodbase/FoodBase_curated.xml', 'r') as f:
    data = f.read()

Bs_data = BeautifulSoup(data, "xml")

b_doc = Bs_data.find_all('document')


In [4]:
print(type(b_doc[0]))

<class 'bs4.element.Tag'>


In [5]:
b_annotations = b_doc[0].find_all("annotation")
print(b_annotations)

[<annotation id="1">
<location length="12" offset="3"/>
<text>cream cheese</text>
<infon key="semantic_tags"> AG.01.e [Dairy produce];AG.01.e.02 [Cheese];AG.01.n [Dishes and prepared food];AG.01.n.18 [Preserve];</infon>
</annotation>, <annotation id="2">
<location length="4" offset="6"/>
<text>beef</text>
<infon key="semantic_tags"> AG.01.d.03 [Beef];</infon>
</annotation>, <annotation id="3">
<location length="6" offset="8"/>
<text>olives</text>
<infon key="semantic_tags"> AG.01.h.01.e [Fruit containing stone];</infon>
</annotation>, <annotation id="4">
<location length="5" offset="10"/>
<text>onion</text>
<infon key="semantic_tags"> AG.01.h.02.e [Onion/leek/garlic];</infon>
</annotation>, <annotation id="5">
<location length="20" offset="13"/>
<text>Worcestershire sauce</text>
<infon key="semantic_tags"> AG.01.h [Fruit and vegetables];AG.01.l.04 [Sauce/dressing];</infon>
</annotation>, <annotation id="6">
<location length="7" offset="63"/>
<text>walnuts</text>
<infon key="semantic_ta

In [40]:
import re

def get_all_occurences(text, doc):
    return [m.start() for m in re.finditer(text, doc)]

In [44]:
data = []
i = 0
for docs in b_doc:
    infon = docs.find("infon", {"key": "full_text"})
    document = infon.get_text().strip()
    data.append([])
    data[i].append(document)
    data[i].append({'entities': []})
    current_end = 0
    # print(document)
    ingredients_list = set()
    for anots in docs.find_all("annotation"):
        word = anots.find("text").get_text()
        ingredients_list.add(word)
    
    for word_ingredient in ingredients_list:
        # print(word_ingredient)
        # print()
        length = len(word_ingredient)
        for start_offset in get_all_occurences(word_ingredient, document):
            end = int(start_offset) + int(length)
            data[i][1]["entities"].append([int(start_offset), end, "INGREDIENT"])
        
        #         place = anots.find("location")
        #         offset = document[current_end:].index(word) + current_end # find the first occurence of this
        #         length = place["length"]
        #         end = int(offset) + int(length)
        #         data[i][1]["entities"].append([int(offset), end, "INGREDIENT"])
        #         current_end = end
    
    i+=1
    

In [46]:
with open("FoodBase_curated_fixed.json", 'w') as outfile:
    json.dump(data, outfile)

In [47]:
with open("FoodBase_curated_fixed.json") as json_file:
    data = json.load(json_file)
print(data[0])

['Mix the cream cheese, beef, olives, onion, and Worcestershire sauce together in a bowl until evenly blended. Keeping the mixture in the bowl, scrape it into a semi-ball shape. Cover, and refrigerate until firm, at least 2 hours. Place a large sheet of waxed paper on a flat surface. Sprinkle with walnuts. Roll the cheese ball in the walnuts until completely covered. Transfer the cheese ball to a serving plate, or rewrap with waxed paper and refrigerate until needed.', {'entities': [[315, 326, 'INGREDIENT'], [381, 392, 'INGREDIENT'], [28, 34, 'INGREDIENT'], [8, 20, 'INGREDIENT'], [47, 67, 'INGREDIENT'], [297, 304, 'INGREDIENT'], [334, 341, 'INGREDIENT'], [36, 41, 'INGREDIENT'], [22, 26, 'INGREDIENT']]}]
